In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [ ]:
df = pd.read_excel('/content/drive/My Drive/Overall_Train_data.xlsx')
#df = df.sample(n=100000, random_state=134)

In [ ]:
df.head()
#df.columns = ['id', 'Verbatim Response', 'IMPROVEMENT', 'LIKE', 'NEUTRAL']
#df.head()

In [ ]:
df['Verbatim_Cleaned'] = df['Verbatim_Cleaned'].str.lower()

In [ ]:
import nltk
nltk.download('punkt')
def tokenize(text):
    text = nltk.word_tokenize(text)
    return text
#df['Sentences'] = df['Verbatim_Cleaned'].apply(tokenize)
#df['n_words'] = [len(i) for i in df['Sentences']]

In [ ]:
#df = df.drop(['Sentences', 'n_words'], axis=1)

In [ ]:
############### PreProcessing ###############
import re 
ONLY_DIGITS = re.compile('^[0-9]+$')  #Remove verbatims which have no words
def preprocess_text(sen):
    # Remove digits only sentences
    #sentence = ONLY_DIGITS.sub('', sen)
    sentence = str(sen).replace("\d+", '')
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    sentence = sentence.lower()

    return sentence

X = df['Verbatim_Cleaned']
x = []
sentences = list(X)
for sen in sentences:
    x.append(preprocess_text(sen))

y = df.iloc[:,5:26].values


unique_words = df['Verbatim_Cleaned'].str.split(expand=True).stack().value_counts() 
print(len(unique_words))

In [ ]:
y

In [ ]:
################   Setting Parameter
#the path for Glove embeddings
GLOVE_DIR = '/content/drive/My Drive/bert/'
# make the max word length to be constant
MAX_WORDS = len(unique_words)
MAX_SEQUENCE_LENGTH = 700
# the percentage of train test split to be applied
VALIDATION_SPLIT = 0.20
# the dimension of vectors to be used
embedding_dim = 100

In [ ]:
#################  Tokenization and Padding ####################
tokenizer  = Tokenizer(num_words = MAX_WORDS)
tokenizer.fit_on_texts(x)
sequences =  tokenizer.texts_to_sequences(x)

word_index = tokenizer.word_index
print("unique words : {}".format(len(word_index)))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

labels = y
print('Shape of data tensor:', data.shape)
print('Shape of label Subcategory tensor:', labels.shape)
print(labels)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/content/drive/My Drive/bert/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data,labels,test_size=0.25, random_state = 342)

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.regularizers import l2
from tensorflow.python.keras.losses import BinaryCrossentropy

model_sentiment = Sequential()
model_sentiment.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=True))
model_sentiment.add(layers.Flatten())
#model.add(layers.Dropout(0.3))
model_sentiment.add(layers.Dense(500, activation='relu'))
model_sentiment.add(layers.Dense(500, activation='relu'))
model_sentiment.add(layers.Dense(21, activation='sigmoid'))
model_sentiment.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['categorical_accuracy'])
model_sentiment.summary()

In [ ]:
history = model_sentiment.fit(X_train, Y_train,
                    epochs=5,
                    verbose=1,
                    validation_split=0.2,
                    batch_size=64)

In [ ]:
loss, accuracy = model.evaluate(X_train, Y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, Y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
predicted = model.predict(X_test)
predicted_train = model.predict(X_train)

In [ ]:
predicted_class = np.round(predicted)
predicted_train = np.round(predicted_train)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
print("Accuracy:", accuracy_score(Y_test, predicted_class))
print("f1_score", f1_score(Y_test, predicted_class, average = 'micro'))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, fbeta_score, precision_score, zero_one_loss, hamming_loss, make_scorer, roc_auc_score, average_precision_score, jaccard_similarity_score, log_loss, precision_recall_fscore_support, recall_score
from sklearn.metrics import coverage_error, label_ranking_average_precision_score, label_ranking_loss 
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    https://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

def print_accuracy_score_train(y_pred):
    print("Hamming score: {}".format(hamming_score(y_train, y_pred)))
    print('Hamming loss: {0}'.format(hamming_loss(y_train, y_pred))) 
    print("Accuracy Score: {}".format(accuracy_score(y_train, y_pred, normalize=True, sample_weight=None)))

def print_accuracy_score_test(y_pred):
    print("Hamming score: {}".format(hamming_score(Y_test, y_pred)))
    print('Hamming loss: {0}'.format(hamming_loss(Y_test, y_pred))) 
    print("Accuracy Score: {}".format(accuracy_score(Y_test, y_pred, normalize=True, sample_weight=None)))
    print("Precision Score: {}".format(precision_score(Y_test, y_pred, average = 'micro')))
    print("Recall Score: {}".format(recall_score(Y_test, y_pred, average = 'micro')))
    print("F1 Score: {}".format(f1_score(Y_test, y_pred, average = 'micro')))
    

In [ ]:
print_accuracy_score_train(predicted_train)
print_accuracy_score_test(predicted_class)

In [ ]:
#### Model for SubCategory ###
model_subcat = Sequential()
model_subcat.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model_subcat.add(layers.Flatten())
#model.add(layers.Dropout(0.3))
model_subcat.add(layers.Dense(500, activation='relu'))
model_subcat.add(layers.Dense(500, activation='relu'))
model_subcat.add(layers.Dense(219, activation='sigmoid'))
model_subcat.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['categorical_accuracy'])
model_subcat.summary()

In [ ]:
history = model_subcat.fit(x_train, y_train_subcat,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_val, y_val_subcat),
                    batch_size=64)

In [ ]:
loss, accuracy = model_subcat.evaluate(x_train, y_train_subcat, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_subcat.evaluate(X_test, Y_test_subcat, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
predicted = model_subcat.predict(X_test)
predicted_train = model_subcat.predict(x_train)

In [ ]:
predicted_class = np.round(predicted)
predicted_train = np.round(predicted_train)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
print("Accuracy:", accuracy_score(Y_test_sent, predicted_class))
print("f1_score", f1_score(Y_test_sent, predicted_class, average = 'micro'))

In [ ]:
test_new = df.sample(n=20000, random_state=343)

In [ ]:
y_test_new = test_new.iloc[:,1:].values
X_test_new = test_new['Verbatim Response']

In [ ]:
############### PreProcessing ###############
import re 
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    sentence = sentence.lower()

    return sentence

x = []
sentences = list(X)
for sen in sentences:
    x.append(preprocess_text(sen))


unique_words = X_test_new.str.split(expand=True).stack().value_counts() 
print(len(unique_words))

In [ ]:
#################  Tokenization and Padding ####################
sequences =  tokenizer.texts_to_sequences(X_test_new)

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [ ]:
predicted = model.predict(data)

In [ ]:
predicted_class = np.round(predicted)

In [ ]:
print("Accuracy:", accuracy_score(y_test_new, predicted_class))
print("f1_score", f1_score(y_test_new, predicted_class, average = 'micro'))

In [ ]:
import sys
import numpy as np
import keras
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D,LSTM, AveragePooling2D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [ ]:
# filter sizes of the different conv layers 
filter_sizes = [1,2,3]
num_filters = 64
embedding_dim = 100
# dropout probability
drop = 0.2
batch_size = 30
epochs = 10

In [ ]:
############# Keras Embedding #######
embedding_layer = Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [ ]:
#################### Model Structure ###############
inputs = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding = embedding_layer(inputs)

print(embedding.shape)
reshape = Reshape((MAX_SEQUENCE_LENGTH,embedding_dim,1))(embedding)
print(reshape.shape)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
#dropout = Dropout(drop)(flatten)
layer_1 = Dense(units=500, activation='relu')(flatten)
#layer_2 = Dense(units=500, activation='relu')(layer_1)
output = Dense(units=21, activation='sigmoid')(layer_1)

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [ ]:
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['categorical_accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train
                       , Y_train
                       , epochs=20
                       , batch_size=64
                       , validation_split = 0.2
                       , verbose=1)

In [ ]:
df = pd.read_excel('/content/drive/My Drive/Improvement_Train.xlsx')
#df = df.sample(n=100000, random_state=134)

df.head()

df['Verbatim_Cleaned'] = df['Verbatim_Cleaned'].str.lower()

X = df['Verbatim_Cleaned']
Y = df.iloc[:,2:]
Y.shape

import nltk
nltk.download('punkt')
def tokenize(text):
    text = nltk.word_tokenize(text)
    return text
#df['Sentences'] = df['Verbatim_Cleaned'].apply(tokenize)
#df['n_words'] = [len(i) for i in df['Sentences']]

#df = df.drop(['Sentences', 'n_words'], axis=1)

############### PreProcessing ###############
import re 
ONLY_DIGITS = re.compile('^[0-9]+$')  #Remove verbatims which have no words
def preprocess_text(sen):
    # Remove digits only sentences
    #sentence = ONLY_DIGITS.sub('', sen)
    sentence = str(sen).replace("\d+", '')
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    
    sentence = sentence.lower()

    return sentence

x = []
sentences = list(X)
for sen in sentences:
    x.append(preprocess_text(sen))

y = df.iloc[:,2:].values

unique_words = df['Verbatim_Cleaned'].str.split(expand=True).stack().value_counts() 
print(len(unique_words))

################   Setting Parameter
#the path for Glove embeddings
GLOVE_DIR = '/content/drive/My Drive/bert/'
# make the max word length to be constant
MAX_WORDS = len(unique_words)
MAX_SEQUENCE_LENGTH = 700
# the percentage of train test split to be applied
VALIDATION_SPLIT = 0.20
# the dimension of vectors to be used
embedding_dim = 100

#################  Tokenization and Padding ####################
tokenizer  = Tokenizer(num_words = MAX_WORDS)
tokenizer.fit_on_texts(x)
sequences =  tokenizer.texts_to_sequences(x)

word_index = tokenizer.word_index
print("unique words : {}".format(len(word_index)))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

labels = y
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
print(labels)

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

# load the whole embedding into memory
embeddings_index = dict()
f = open('/content/drive/My Drive/bert/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data,labels,test_size=0.25, random_state = 342)
x_train, x_val, y_train, y_val = train_test_split(X_train,Y_train,test_size=0.2, random_state = 34)

In [ ]:
model_like = Sequential()
model_like.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))
model_like.add(layers.Flatten())
#model.add(layers.Dropout(0.3))
model_like.add(layers.Dense(500, activation='relu'))
model_like.add(layers.Dense(500, activation='relu'))
model_like.add(layers.Dense(161, activation='sigmoid'))
model_like.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()

history = model_like.fit(x_train, y_train,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    batch_size=64)

In [ ]:
loss, accuracy = model_like.evaluate(x_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model_like.evaluate(X_test, Y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

predicted = model_like.predict(X_test)
predicted_train = model_like.predict(x_train)

In [ ]:
predicted_class = np.round(predicted)
predicted_train = np.round(predicted_train)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, fbeta_score, precision_score, zero_one_loss, hamming_loss, make_scorer, roc_auc_score, average_precision_score, jaccard_similarity_score, log_loss, precision_recall_fscore_support, recall_score
from sklearn.metrics import coverage_error, label_ranking_average_precision_score, label_ranking_loss 
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    https://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

def print_accuracy_score_train(y_pred):
    print("Hamming score: {}".format(hamming_score(y_train, y_pred)))
    print('Hamming loss: {0}'.format(hamming_loss(y_train, y_pred))) 
    print("Accuracy Score: {}".format(accuracy_score(y_train, y_pred, normalize=True, sample_weight=None)))

def print_accuracy_score_test(y_pred):
    print("Hamming score: {}".format(hamming_score(Y_test, y_pred)))
    print('Hamming loss: {0}'.format(hamming_loss(Y_test, y_pred))) 
    print("Accuracy Score: {}".format(accuracy_score(Y_test, y_pred, normalize=True, sample_weight=None)))
    print("Precision Score: {}".format(precision_score(Y_test, y_pred, average = 'micro')))
    print("Recall Score: {}".format(recall_score(Y_test, y_pred, average = 'micro')))
    print("F1 Score: {}".format(f1_score(Y_test, y_pred, average = 'micro')))

print_accuracy_score_train(predicted_train)
print_accuracy_score_test(predicted_class)

In [ ]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=MAX_SEQUENCE_LENGTH, 
                           trainable=False))

model.add(LSTM(128))
model.add(Dense(units=36, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['categorical_accuracy'])
model.summary()

In [ ]:
history = model.fit(X_train, Y_train,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_test, Y_test),
                    batch_size=64)